In [2]:
!conda install -c conda-forge xlrd --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    xlrd-1.2.0                 |             py_0         108 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    scip

In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from IPython.display import display_html
import json # library to handle JSON files
from bs4 import BeautifulSoup
import re
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Preparing crime data

In [4]:
# read in the excel table
xls = pd.ExcelFile('crimeBerlin.xlsx')
df1 = pd.read_excel(xls, 'Titel')
df2 = pd.read_excel(xls, 'Inhaltsverzeichnis')

In [5]:
# read the single sheets into a data frame
# rename some colums and drop some rows
df3 = pd.read_excel(xls, 'Fallzahlen_2018') # up to 2018
df3.drop(df3.tail(2).index,inplace=True) # drop last 2 rows
df3.drop(df3.head(4).index,inplace=True) # drop first 4 rows
newcols=['Borough','Neighborhood', 'Total records', 'Robbery', 'Street robbery', \
      'Total physical injury', 'Serious bodily harm', 'Coercion', 'Total theft', \
      'Bicycle theft', 'Burglary', 'Property damage', 'Drug offenses']
df3 = df3.drop(df3.columns[[9,10,13,14,16,18]], axis=1)
df3.columns=newcols
df3.reset_index(drop=True, inplace=True)
df3.head()

,Borough,Neighborhood,Total records,Robbery,Street robbery,Total physical injury,Serious bodily harm,Coercion,Total theft,Bicycle theft,Burglary,Property damage,Drug offenses
0,010000,Mitte,85227,733,449,7450,1854,2068,37802,4173,775,5528,3792
1,010111,Tiergarten Süd,5171,70,58,416,125,103,2540,310,39,284,273
2,010112,Regierungsviertel,9146,34,16,519,110,114,3883,372,35,407,133
3,010113,Alexanderplatz,19275,130,80,1531,366,309,10144,880,133,1036,971
4,010114,Brunnenstraße Süd,4207,26,14,286,60,69,1817,314,56,442,69


In [6]:
# replace LOR-keys with the corresponding borough names
lorkeys=['010000','020000','030000','040000','050000','060000','070000','080000','090000','100000','110000','120000']
myIndex=[]
for tkey in lorkeys:
    df3ind=df3.index[df3['Borough'] == tkey].to_list()
    myIndex.append(df3ind[0])
myIndex = [ int(x) for x in myIndex ]
myBorough=['Mitte','Friedrichshain-Kreuzberg','Pankow', 'Charlottenburg-Wilmersdorf', 'Spandau', 'Steglitz-Zehlendorf', \
          'Tempelhof-Schöneberg', 'Neukölln', 'Treptow-Köpenick', 'Marzahn-Hellersdorf', 'Lichtenberg','Reinickendorf']
myb=df3['Borough'].astype(int).to_list()
start=10000
k=0
myBor=[]
for i in myb:
    if i < start+10000:
        myBor.append(myBorough[k])
    else:
        k=k+1
        myBor.append(myBorough[k])
        start=start+10000
        
df3['Borough']=myBor
df3.drop(myIndex,axis=0, inplace=True)
df3.head()

,Borough,Neighborhood,Total records,Robbery,Street robbery,Total physical injury,Serious bodily harm,Coercion,Total theft,Bicycle theft,Burglary,Property damage,Drug offenses
1,Mitte,Tiergarten Süd,5171,70,58,416,125,103,2540,310,39,284,273
2,Mitte,Regierungsviertel,9146,34,16,519,110,114,3883,372,35,407,133
3,Mitte,Alexanderplatz,19275,130,80,1531,366,309,10144,880,133,1036,971
4,Mitte,Brunnenstraße Süd,4207,26,14,286,60,69,1817,314,56,442,69
5,Mitte,Moabit West,7257,71,38,653,164,215,2470,447,83,496,435


In [7]:
# writing crime data into csv file
df3.to_csv("berlincrime2018.csv")

#### Preparing school data

In [8]:
# load the web site
url='https://www.gymnasium-berlin.net/abiturdaten/2018'
content = requests.get(url)
soup = BeautifulSoup(content.text, 'html.parser')

myList=[]

for br in soup.find_all("br"):
        br.replace_with("\n\n")
        
rows = soup.find_all('tr') # Extract and return first occurrence of tr
for row in rows:
    text = str(row.get_text())
    b=text.split("\n\n")
    myList.append(b)

In [9]:
# create the data frame
df1=pd.DataFrame(myList)
df1.columns=['T','ID','School','Neighborhood','Grade']
df1.drop(df1.head(1).index,inplace=True)
df1.drop(columns=['T','ID'],inplace=True)
df1.drop(df1.tail(11).index,inplace=True)
df1 = df1.replace('\n','', regex=True)
df1.head()

,School,Neighborhood,Grade
1,Französisches Gymnasium,Tiergarten,"1,67"
2,Heinrich-Hertz-Gymnasium,Friedrichshain,"1,83"
3,Georg-Friedrich-Händel-Gymnasium,Friedrichshain,"1,89"
4,Rosa-Luxemburg-Gymnasium,Pankow,"1,90"
5,Arndt-Gymnasium Dahlem,Dahlem,"1,94"


In [10]:
url='https://www.gymnasium-berlin.net/adressliste'
content = requests.get(url)
soup = BeautifulSoup(content.text, 'html.parser')

myList=[]

for br in soup.find_all("br"):
        br.replace_with("\n\n")
        
rows = soup.find_all('tr') # Extract and return first occurrence of tr
for row in rows:
    text = str(row.get_text())
    b=text.split("\n\n")
    myList.append(b)

In [11]:
df0=pd.DataFrame(myList)
df0.drop(df0.tail(7).index,inplace=True)
df0.columns=['ID','School','Address','Postcode']
df0.drop(columns='ID',inplace=True)
df0 = df0.replace('Berlin','', regex=True)
df0 = df0.replace('\n','', regex=True)
df0.head()

,School,Address,Postcode
0,Albert-Einstein-Gymnasium,Parchimer Allee 109,12359
1,Albert-Schweitzer-Gymnasium,Karl-Marx-Straße 14,12043
2,Albrecht-Dürer-Gymnasium,Emser Straße 132-137,12051
3,Alexander-von-Humboldt-Gymnasium,Oberspreestraße 173-181,12555
4,Andreas-Gymnasium,Koppenstraße 76,10243


In [12]:
df0.set_index('School')
df1.set_index('School')
df_sc = pd.merge(df0, df1, on=['School'])
df_sc.drop(columns=['School','Address'],inplace=True)
df_sc.reset_index(drop=True, inplace=True)
df_sc.head()

,Postcode,Neighborhood,Grade
0,12359,Britz,"2,30"
1,12043,Neukölln,"2,63"
2,12051,Neukölln,"2,60"
3,12555,Köpenick,"2,33"
4,10243,Friedrichshain,"2,55"


In [13]:
# writing school data into csv file
df_sc.to_csv("berlinschool2019.csv")

#### Preparing rental costs data

In [14]:
# load the web site
url='https://www.wohnungsboerse.net/mietspiegel-Berlin/2825'
content = requests.get(url)
soup = BeautifulSoup(content.text, 'html.parser')

In [15]:
#extract the corresponing table
names=[]
for tr in soup.find_all('tr'):
    td = [td for td in tr.stripped_strings]
    if len(td)  > 2:
        a=td[0]
        b=""
        b=b.join(a)    #b=str(a[0]).strip('[]')
        names.append(b)

In [16]:
rents=[]
for tr in soup.find_all('tr'):
    td = [td for td in tr.stripped_strings]
    if len(td)  > 2:
        a=td[2]
        b=""
        b=b.join(a)
        rents.append(b)

In [17]:
# create the data frame
df2 = pd.DataFrame(
    {'Total Neighborhood': names[29:115],
     'Rental costs': rents[29:115]
    })
df2 = df2.replace('\xa0€','', regex=True) # replace the euro sign
df2 = df2.replace('\(','', regex=True)
df2 = df2.replace('\)','', regex=True)
df2.head()

,Total Neighborhood,Rental costs
0,Adlershof Treptow,"12,51"
1,Alt-Hohenschönhausen Hohenschönhausen,"10,76"
2,Alt-Treptow,"12,90"
3,Altglienicke Treptow,"9,95"
4,Baumschulenweg Treptow,"10,82"


In [18]:
newdf = df2["Total Neighborhood"].str.split(" ", n = 1, expand = True)
newdf.head(3)

,0,1
0,Adlershof,Treptow
1,Alt-Hohenschönhausen,Hohenschönhausen
2,Alt-Treptow,None


In [19]:
df2['Neighborhood']=newdf[0].to_list()
df2.head()

,Total Neighborhood,Rental costs,Neighborhood
0,Adlershof Treptow,"12,51",Adlershof
1,Alt-Hohenschönhausen Hohenschönhausen,"10,76",Alt-Hohenschönhausen
2,Alt-Treptow,"12,90",Alt-Treptow
3,Altglienicke Treptow,"9,95",Altglienicke
4,Baumschulenweg Treptow,"10,82",Baumschulenweg


In [20]:
df2.drop(columns=['Total Neighborhood'],inplace=True)
#df2['Rental costs']=pd.to_numeric(df2['Rental costs'],errors='coerce')
cols = df2.columns.tolist()
cols = cols[-1:] + cols[:-1]
df2 = df2[cols]
df2.head()

,Neighborhood,Rental costs
0,Adlershof,"12,51"
1,Alt-Hohenschönhausen,"10,76"
2,Alt-Treptow,"12,90"
3,Altglienicke,"9,95"
4,Baumschulenweg,"10,82"


['Neighborhood', 'Rental costs']

In [21]:
# writing crime data into csv file
df2.to_csv("berlinrenatlcosts2019.csv")

#### Preparing venue data

In [75]:
# getting post codes and Neighborhoods by a direck loading into panda data frame
url='https://de.wikipedia.org/wiki/Verwaltungsgliederung_Berlins'
dfs = pd.read_html(url, header=0)
dfl=dfs[2]
dfl.columns=['T1','Neighborhood', 'Borough', 'T2' ,'T3', 'T4']
dfl.drop(columns=['T1','T2' ,'T3', 'T4'],inplace=True)
dfl.head()

,Neighborhood,Borough
0,Mitte,Mitte
1,Moabit,Mitte
2,Hansaviertel,Mitte
3,Tiergarten,Mitte
4,Wedding,Mitte


In [76]:
dfp=pd.read_csv('plz.csv')
dfp.head(5)

,PLZ,Stadtteil
0,10115-10435,Mitte
1,10119-10439,Prenzlauer Berg
2,10243-10249,Friedrichshain
3,10315-10319,Friedrichsfelde
4,10315-10369,Lichtenberg


In [77]:
dfn = dfp["PLZ"].str.split("-", n = 1, expand = True)
dfp['Postcode']=dfn[0].to_list()
dfp['Postcode']=dfp['Postcode'].astype(int)
dfp.drop(columns=['PLZ'],inplace=True)
dfp.columns=['Neighborhood','Postcode']
dfp = dfp.replace(' ','', regex=True)
dfl = dfl.replace(' ','', regex=True)
dfp.head()

,Neighborhood,Postcode
0,Mitte,10115
1,PrenzlauerBerg,10119
2,Friedrichshain,10243
3,Friedrichsfelde,10315
4,Lichtenberg,10315


In [72]:
i1=dfp['Postcode'].to_list()
l1=dfp['Neighborhood'].to_list()
l2=dfl['Neighborhood'].to_list()
Z = [x for _,x in sorted(zip(l2,l1))]
print(Z)

In [78]:
print(dfp.shape, dfl.shape)
dfl.reset_index(drop=True, inplace=True)
dfp.reset_index(drop=True, inplace=True)
dfl.set_index('Neighborhood')
dfp.set_index('Neighborhood')
result = pd.merge(dfp, dfl,  on=['Neighborhood'])
result.head(40)

(96, 2) (96, 2)


,Neighborhood,Postcode,Borough
0,Mitte,10115,Mitte
1,PrenzlauerBerg,10119,Pankow
2,Friedrichshain,10243,Friedrichshain-Kreuzberg
3,Friedrichsfelde,10315,Lichtenberg
4,Lichtenberg,10315,Lichtenberg
5,Rummelsburg,10317,Lichtenberg
6,Karlshorst,10318,Lichtenberg
7,Fennpfuhl,10367,Lichtenberg
8,Pankow,10439,Pankow
9,Moabit,10551,Mitte


In [83]:
cols = result.columns.tolist()
cols = cols[-1:] + cols[:-1]
result = result[cols]
result.head(1)

,Borough,Neighborhood,Postcode
0,Mitte,Mitte,10115


In [84]:
# writing to a csv file
dfp.to_csv("berlinpostal.csv")

In [85]:
# reading an csv file with coordinates
df4= pd.read_csv('Locations.csv',sep=';')
df4.drop(columns=['Verwaltungszusammenschluss','Regierungsbezirk','ID',\
                  'Landkreis','Staat','Bundesland'],inplace=True)
df4=df4[df4['Ort']=='Berlin']
df4.head()

,Ort,Longitude,Latitude,Postcode
7748,Berlin,13.387224,52.533707,10115
7749,Berlin,13.390193,52.518746,10117
7750,Berlin,13.407149,52.532666,10119
7751,Berlin,13.412203,52.523474,10178
7752,Berlin,13.419699,52.514591,10179


In [87]:
# merge coordinates and Neighborhood frames on Postcode
df4.reset_index(drop=True, inplace=True)
df4.drop(columns=['Ort'], inplace=True)
df4.head()

,Longitude,Latitude,Postcode
0,13.387224,52.533707,10115
1,13.390193,52.518746,10117
2,13.407149,52.532666,10119
3,13.412203,52.523474,10178
4,13.419699,52.514591,10179


In [88]:
result2 = pd.merge(result, df4, on=['Postcode'])
result2.head(3)

,Borough,Neighborhood,Postcode,Longitude,Latitude
0,Mitte,Mitte,10115,13.387224,52.533707
1,Pankow,PrenzlauerBerg,10119,13.407149,52.532666
2,Friedrichshain-Kreuzberg,Friedrichshain,10243,13.441956,52.514918


In [92]:
result2.drop(columns=['Postcode'], inplace=True)
result2.head(4)

,Borough,Neighborhood,Longitude,Latitude
0,Mitte,Mitte,13.387224,52.533707
1,Pankow,PrenzlauerBerg,13.407149,52.532666
2,Friedrichshain-Kreuzberg,Friedrichshain,13.441956,52.514918
3,Lichtenberg,Friedrichsfelde,13.518035,52.511031


In [93]:
# writing to a csv file coordinates and neighborhoods
result2.to_csv("berlin_neighb_coords.csv")

In [99]:
# create map using latitude and longitude values
# Longitude: 13.283333
# Latitude: 52.500000
latitude  = 52.500000
longitude = 13.283333
address = 'Berlin, DE'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Berlin are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Berlin are 52.5170365, 13.3888599.


In [100]:
# take a mitte borough
mitte_data = result2[result2['Borough'] == 'Charlottenburg-Wilmersdorf'].reset_index(drop=True)
mitte_data.head()

,Borough,Neighborhood,Longitude,Latitude
0,Charlottenburg-Wilmersdorf,Charlottenburg,13.307670,52.516531
1,Charlottenburg-Wilmersdorf,Charlottenburg-Nord,13.308494,52.529399
2,Charlottenburg-Wilmersdorf,Wilmersdorf,13.317009,52.498305
3,Charlottenburg-Wilmersdorf,Halensee,13.306143,52.495449
4,Charlottenburg-Wilmersdorf,Grunewald,13.294385,52.499381


In [101]:
# create map of Berlin-Mitte using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(mitte_data['Latitude'],mitte_data['Longitude'], mitte_data['Borough'],mitte_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)  
    
map_berlin

In [102]:
CLIENT_ID = 'QR03XDVIW1X5MRLXKYCNVVGBH5Q0PLQ4NZMLAPDH345MGEXY' # your Foursquare ID
CLIENT_SECRET = 'T4BALEMPTZJHYTSV2PCTPMZEUHSHTJUXBNZU1JPMNMGRKN5D' # your Foursquare Secret
VERSION = '20180604'
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

# coordinates of Charlottenburg-Wilmersdorf
neighborhood_latitude  = 52.500000  # neighborhood latitude value
neighborhood_longitude = 13.283333 # neighborhood longitude valu

In [103]:
# create the GET request URL. 
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QR03XDVIW1X5MRLXKYCNVVGBH5Q0PLQ4NZMLAPDH345MGEXY&client_secret=T4BALEMPTZJHYTSV2PCTPMZEUHSHTJUXBNZU1JPMNMGRKN5D&v=20180604&ll=52.5,13.283333&radius=500&limit=100'

In [104]:
results = requests.get(url).json()

In [105]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [106]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Fressnapf,Pet Store,52.499663,13.288616
1,Hotel Kronprinz,Hotel,52.497091,13.287195
2,Gleis 3/4,Platform,52.500946,13.283662
3,"Reifen-Müller, Georg Müller",Automotive Shop,52.501168,13.281750
4,Ditsch,Bakery,52.500522,13.284126


In [107]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.
